# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Extraordinaria (2 de julio de 2021)
## Curso 2020-2021

# Definiciones auxiliares

In [2]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas funciones sobre tipos estándar de la librería de Scala

In [3]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [4]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
type Or[P, Q] = Either[P, Q]
type And[P, Q] = (P, Q)

defined type Not
defined type <=>
defined type Or
defined type And

# Ejercicio 1 (variante 1)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que las siguientes proposiciones representan teoremas de la lógica intuicionista: 

$\vdash \neg (p \vee q) \rightarrow \neg p \wedge \neg q$


In [103]:
def proof[P, Q](p1: Not[Either[P, Q]]): (Not[P], Not[Q]) = 
    ((p:P) => p1(Left(p)), 
     (q: Q) => p1(Right(q)))

defined function proof

$\vdash \neg p \wedge \neg q \rightarrow \neg (p \vee q)$

In [9]:
def proof[P, Q](p1: (Not[P], Not[Q])): Not[Either[P, Q]]  = 
    { 
        case Left(p) => p1._1(p)
        case Right(q) => p1._2(q)
    }

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$\vdash \neg(\neg p \vee \neg q) \rightarrow p \wedge q$

Supóngase para ello que la ley del tercio excluso se cumple para las variables proposicionales $p$ y $q$, es decir, que las fórmulas $p ∨ ¬p$ y $q ∨ ¬q$  pueden utilizarse como premisas.


In [11]:
def proof[P, Q](pnp: Either[P, Not[P]], qnq: Either[Q, Not[Q]]): Not[Either[Not[P], Not[Q]]] => (P, Q) = 
    (f: Either[Not[P], Not[Q]] => Nothing) => 
        (pnp, qnq) match {
            case (Left(p), Left(q)) => (p, q)
            case (Left(p), Right(nq)) => (p, f(Right(nq)))
            case (Right(np), Left(q)) => (f(Left(np)), q)
            case (Right(np), Right(nq)) => (f(Left(np)), f(Right(nq)))
        }

defined function proof

# Ejercicio 2 
__(2 puntos)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$, $Y$ y $Z$: 

$(Y*Z)^X \cong Y^X*Z^X$

In [79]:
def from[X, Y, Z](f: X => (Y, Z)): (X => Y, X => Z) = 
    (x => f(x)._1, x => f(x)._2)


defined function from

In [80]:
def to[X, Y, Z](t: (X => Y, X => Z)): X => (Y, Z)  = 
    x => (t._1(x), t._2(x))


defined function to

# Ejercicio 3
__(5 puntos)__

La función `split` recibe una lista de pares y devuelve un par de listas formadas por los elementos de la primera y segunda componentes, respectivamente. El comportamiento de la función se ilustra en el siguiente test unitario:


In [45]:
class SplitTest(
    split: List[(Int, Boolean)] => (List[Int], List[Boolean])
) extends FlatSpec with Matchers{
    "split" should "work" in {
        split(List()) shouldBe (List(), List())
        split(List((1, true), (2, false), (3, true))) shouldBe (List(1,2,3), List(true, false, true))
        split(List((0, true), (0, true))) shouldBe (List(0,0), List(true, true))
    }
}

defined class SplitTest

__a) (1 punto)__ Implementa la función `split` de manera recursiva (no final).

In [95]:
def splitR[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l match {
        case Nil => (Nil, Nil)
        case (a,b) :: tail => 
            val (la, lb) = split(tail)
            (a :: la, b :: lb)
    }

defined function splitR

In [96]:
run(new SplitTest(splitR))

cmd44$Helper$SplitTest:
split
- should work


__b) (1 punto)__ Implementa la función `split` utilizando __`foldRight`__.

In [97]:
def splitFR[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l.foldRight((List[A](), List[B]())){
        case ((a,b), (la, lb)) => 
            (a :: la, b :: lb)
    }

defined function splitFR

In [98]:
run(new SplitTest(splitFR))

cmd44$Helper$SplitTest:
split
- should work


__c) (1 punto)__ Implementa la función `split` mediante recursión por cola (o final):

In [99]:
def splitTR[A, B](l: List[(A, B)]): (List[A], List[B]) = {
    def aux(acc: (List[A], List[B]), l: List[(A, B)]): (List[A], List[B]) = 
        (acc, l) match {
            case ((la, lb), Nil) => (la.reverse, lb.reverse)
            case ((la, lb), (a, b) :: tail) => aux((a :: la, b :: lb), tail)
        }
    aux((List[A](), List[B]()), l)
}

defined function splitTR

In [100]:
run(new SplitTest(splitTR))

cmd44$Helper$SplitTest:
split
- should work


__d) (1 punto)__ Implementa la función `split` utilizando __`foldLeft`__.

In [101]:
def splitFL[A, B](l: List[(A, B)]): (List[A], List[B]) = 
    l.foldLeft((List[A](), List[B]())){
        case ((la, lb), (a,b)) => 
            (a :: la, b :: lb)
    } match {
        case (la, lb) => (la.reverse, lb.reverse)
    }

defined function splitFL

In [102]:
run(new SplitTest(splitFL))

cmd44$Helper$SplitTest:
split
- should work


__e) (1 punto)__ Dada una lista de puntos del plano representados mediante coordenadas polares, se desean obtener los valores de las abcisas y ordenadas de sus coordenadas rectangulares. Concretamente, dados los tipos de datos:

In [78]:
case class Polar(radius: Double, angle: Double)
type Rectangular = (Double, Double)

defined class Polar
defined type Rectangular

y la función de conversión entre coordenadas polares y rectangulares: 

In [67]:
import scala.math.{Pi, sin, cos}

def from(p: Polar): Rectangular = 
    (p.radius * cos(p.angle), p.radius * sin(p.angle))

import scala.math.{Pi, sin, cos}


defined function from

se desea implementar una función `components` que satisfaga el siguiente test unitario:

In [75]:
class ComponentsTest(
    components: List[Polar] => (List[Double], List[Double])
) extends FlatSpec with Matchers{
    "components" should "work" in {
        components(List(Polar(1,0), Polar(2,Pi/4), Polar(3,Pi/2), Polar(4,Pi))) shouldBe 
            (List(cos(0), 2*cos(Pi/4), 3*cos(Pi/2), 4*cos(Pi)), 
            List(sin(0), 2*sin(Pi/4), 3*sin(Pi/2), 4*sin(Pi)))
    }
}

defined class ComponentsTest

Implementa la función `components` utilizando la función `split` y alguna de las funciones de orden superior explicadas en clase (`map`, `flatMap`, `foldLeft`, `foldRight`, etc.), de tal forma que la implementación sea lo más concisa posible.

In [76]:
def components(l: List[Polar]): (List[Double], List[Double]) = 
    split(l.map(from))

defined function components

In [77]:
run(new ComponentsTest(components))

cmd74$Helper$ComponentsTest:
components
- should work
